In [1]:
import numpy as np
import torch
from mocapact import utils
from mocapact.envs import tracking
from dm_control.locomotion.tasks.reference_pose import types
from mocapact.distillation import model

/Users/steve/.pyenv/versions/3.10.10/lib/python3.10/site-packages/gym/envs/registration.py:250: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for plugin in metadata.entry_points().get(entry_point, []):
/Users/steve/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/steve/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/Users/steve/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
/Users

In [2]:
from IPython.display import HTML
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


def visualize_images(images, show=True, to_file = None):
    fig, ax = plt.subplots()
    x = ax.imshow(images[0])

    def update(frame):
        x.set_data(images[frame])
        return x

    ani = FuncAnimation(fig, update, frames=range(len(images)), interval=50)
    if to_file is not None:
        ani.save(to_file, writer='imagemagick', fps=30)
    if show:
        display(HTML(ani.to_html5_video()))
    plt.close()

In [3]:
snippets = [
    'CMU_016_15-0-127',  # walking
    'CMU_016_55-0-47'    # running
]

env = tracking.MocapTrackingGymEnv(
    utils.make_clip_collection(snippets),
    ref_steps=list(range(1, 6)),
    act_noise=0.0,
    task_kwargs=dict(
        reward_type='comic',
        min_steps=9,
        ghost_offset=np.array([1., 0., 0.]),
        always_init_at_clip_start=False,
        termination_error_threshold=0.3
    )
)

In [4]:
model_path = "multiclip/walkrun_rnn/model/last.ckpt"
policy = model.RnnPolicy.load_from_checkpoint(model_path, map_location="cpu")

In [5]:
ims = []
with torch.no_grad():
    time_step = env._env.reset()

    while not time_step.last():
        if time_step.step_type == 0:
            state = None
        obs = env.get_observation(time_step)
        action, state = policy.predict(obs, state, deterministic=False)
        time_step = env._env.step(action)
        img = env.render()
        ims.append(img)

In [9]:
visualize_images(ims, to_file="walkrun_example.gif")